In [ ]:
import numpy as np
import os.path as op
import pandas as pd
import mne
from mne import setup_volume_source_space
from mne.beamformer import apply_lcmv_cov, make_lcmv
from mne.cov import compute_covariance



In [ ]:
#dictionary of subejcts and their raw recordings 
subjects = {"patient_X" : "fif file path"}

In [ ]:
#dictionary of coregistration files
coregs_files = {"patient_X" : "trans file path"}

In [ ]:
#dictionaries of epochs files
all_epochs_files = {"patient_X" : "file path"}

high_epochs_files = {"patient_X" : " file path"}

low_epochs_files = {"patient_X" : " file path"}

In [ ]:
for subject, path in subjects.items():
    raw_fname = op.abspath( subjects[subject])
    raw = mne.io.read_raw_fif(raw_fname)
    info = raw.info

    base_subject = subject.split("_part")[0] 
    
  
    
    data_path = op.abspath("path")
    subjects_dir = op.join(data_path, "freesurfer", "recons")
    coreg_fname = op.abspath(coregs_files[subject])
    coreg = mne.read_trans(coreg_fname)

    #improt 3 sets of epochs
    all_epochs_fname = op.abspath(all_epochs_files[subject])
    all_epochs = mne.read_epochs(all_epochs_fname)

    high_epochs_fname = op.abspath(high_epochs_files[subject])
    high_epochs = mne.read_epochs(high_epochs_fname)

    low_epochs_fname = op.abspath(low_epochs_files[subject])
    low_epochs = mne.read_epochs(low_epochs_fname)


    #filter in the ripple band
    all_epochs = all_epochs.filter(80, 150)
    high_epochs = high_epochs.filter(80, 150)
    low_epochs = low_epochs.filter(80, 150)

    #prepare source space
    surface = op.join(subjects_dir, base_subject , "bem" , "inner_skull.surf")
    src = mne.setup_volume_source_space(
    base_subject, subjects_dir=subjects_dir, pos=10.0, surface=surface, add_interpolator=False
    )  

    conductivity = (0.3,)
    model = mne.make_bem_model(
    subject=base_subject, ico=4, conductivity=conductivity, subjects_dir=subjects_dir
    )
    bem = mne.make_bem_solution(model)

    #compute forward solution 
    fwd = mne.make_forward_solution(
        raw_fname,
        trans=coreg,
        src=src,
        bem= bem,
        meg=True,
        eeg=False,
        mindist=5.0,
        n_jobs=None,
        verbose=True,
        )
   #prepare covariance matrix
    rank = mne.compute_rank(all_epochs, tol=1e-6, tol_kind="relative")
    active_win = (-0.1, 0.1)
    baseline_win = (-0.1, 0.1)
    common_win = (-0.1, 0.1)

    #epochs of no SWRs
    baseline_cov = compute_covariance(
        low_epochs,
        tmin=baseline_win[0],
        tmax=baseline_win[1],
        method="shrunk",
        rank=rank,
        verbose=True,
    )
    #epochs of SWRs
    active_cov = compute_covariance(
        high_epochs,
        tmin=active_win[0],
        tmax=active_win[1],
        method="shrunk",
        rank=rank,
        verbose=True,
    )

    #epochs of both
    cov = compute_covariance(
        all_epochs,
        tmin=common_win[0],
        tmax=common_win[1],
        method="shrunk",
        rank=rank,
        verbose=True,
    )

    noise_cov = mne.make_ad_hoc_cov(all_epochs.info) 

    # Weighted averaging is already in the addition of covariance objects.
    cov.plot(all_epochs.info)

    #calculate beamformer weights
    filters = make_lcmv(
            all_epochs.info, fwd, cov,
            reg=0.05,noise_cov=noise_cov,
            pick_ori="max-power", 
            weight_norm= None
        )


    stc_base = apply_lcmv_cov(baseline_cov, filters)
    stc_act = apply_lcmv_cov(active_cov, filters)


    stc_act.data = stc_act.transform_data(np.log10)
    stc_base.data = stc_base.transform_data(np.log10)

    #get the difference between the two conditions
    stc_dB = stc_act - stc_base

    
    stc_dB.plot(subject=base_subject,subjects_dir=subjects_dir,src=src,mode='glass_brain')

    #save 
    stc_dB.save_as_volume(f'path_{subject}.nii', src,  overwrite=True)
    #also can save a stc file
    stc_dB.save(f'path_{subject}', overwrite = True)